In [1]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO
#google_sheet_url = 'https://docs.google.com/spreadsheets/d/1QwSweXB3N7hOAIAeE2Y-yFkSbPGTG0w-zqVHE330sTE/export?format=csv'
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1IsHvruE7NKs7APPOv0NKfh8R-HPhP5717nY7MmlYF3A/export?format=csv'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_csv(BytesIO(data))
local_index = []
df.head(5)





,Booth,CustomerCode,ZoneName,SupplyPointCode,CustomerName,Sale Date,DOB,Address,ContactAddress,IDProof,...,TM500,SM250,SM500,CM500,FCM500,TM500.1,SM250.1,SM500.1,CM500.1,FCM500.1
0,139,CBE219308,NaN,139,SIVAKAMI,5/8/2020,1945-08-14 00:00:03,NaN,Address WIll be filled later,Ration Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1899,CBE223235,NaN,1899,Ragunathan.R,5/8/2020,1945-08-14 00:00:03,NaN,Address WIll be filled later,Ration Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2211,CBE127019,NaN,2211,G.SHAMUGAVADIVELAN,5/8/2020,1945-08-14 00:00:03,NaN,Address WIll be filled later,Ration Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def isNaN(num):
    return num != num

In [4]:
current_cust_code = "None"
jump="dont_skip"

In [4]:
# Loops saves the last seen id from spreadsheet. 
# If jump is set to "skip" it will not process any line until it comes across that line
# If you want to swtich off that behaviour set jump to "dont_skip"
# jump="skip"

In [5]:
cust_type = ICustomerType.objects.get(name='Customer')
cust_user_type = UserType.objects.get(name='Icustomer')
union = Union.objects.get(id=1)
gender = Gender.objects.get(id=1)
taluk = Taluk.objects.get(id=4)
district = District.objects.get(id=1)
state = State.objects.get(id=1)

count = 0


for index, row in df.iterrows():
    count = count+1
    cust_code = str(row['CustomerCode'])

    
    if cust_code == current_cust_code:
        jump = "dont_skip"

    if jump == "skip":
        print(cust_code, 'Count:', count, "\t Skipped")
        continue
        
    current_cust_code = cust_code
    cust_name = row['CustomerName']
    first_name = cust_name
    last_name = ""
#     if '.' in cust_name:
#         [first_name, last_name] = cust_name.split('.', 1)

    
    cust_dob = row['DOB']
    cust_dob = '1996-06-06'

    cust_address = row['Address']
    cust_contact_address = row['ContactAddress']
    cust_id_type = row['IDProof']
    cust_id_value = row['IDProofNo']
    
    # contact info
    # Fill mobile number. if it is landline no (7 len) add 422 to it and fill
    # if there are two mobile numbers, add first to mobile sencond to alternate_phone.
    # if alternate mobile nummber and landline number exists, give pref to alternate_phone
    cust_phone = row['Telephone No']
    cust_mobile = row['MobileNo']
    cust_mobile_alt =""
    if ',' in str(cust_mobile):
        cust_mobile, cust_mobile_alt = str(cust_mobile).split(',')
    
    if '/' in str(cust_mobile):
        cust_mobile, cust_mobile_alt = str(cust_mobile).split('/')
    
    if len(cust_mobile_alt) > 7:
        cust_phone = cust_mobile_alt
    if len(str(cust_mobile)) == 7:
        cust_mobile = int('0422' + str(cust_mobile))
    if not isinstance(cust_phone, float):
        if len(str(cust_phone)) == 7:
            cust_phone = int('0422' + str(cust_phone))
    

    # delivery point
    bus_name = row['SupplyPointName']
    bus_code = row['SupplyPointCode'] 
        
    
    zone_name = row['ZoneName']
    #zone_code = row['ZoneCode']
    #route_name = row['RouteName']
    #route_code= row['RouteCode']
    
    # Create an User table entry for this iCustomer
    cust_user, created =  User.objects.get_or_create(
        username=cust_code,
        defaults = {
            'password':make_password(cust_mobile),
            'first_name': first_name,
            'last_name': 'Kultivate-Aug220',
            'is_staff': False,
            'is_active': True,
            'is_superuser': False,
        }
        
    )
    if created:
        print(cust_user.first_name, '(', cust_code, ')', 'Count:', count, "\n\tuser created")
    else:
        print(cust_user.first_name,  '(', cust_code, ')','Count:', count, "\n\tuser existed already")
        

    
    # Create new User Profile for this iCustomer
    cust_userprofile, created = UserProfile.objects.get_or_create(
        user=cust_user,
        defaults = {
            'union':union,
            'user_type':cust_user_type,
            'street':cust_address,
            'gender':gender,
            'taluk':taluk,
            'district':district,
            'state':state,
            'mobile':cust_mobile
            }
    )
    
    if created:
        print("\tuser profile created")
    else:
        print("\tuser profile existed already")
    
    if cust_phone:
        cust_userprofile.phone = cust_phone
        cust_userprofile.save()
        
    if cust_mobile_alt:
        cust_userprofile.alternate_mobile = cust_mobile_alt
        cust_userprofile.save()
        
    
    
    # Finally fill in the iCustomer table
    
    kyc_idproof_document_type = KycIDProofDocumentType.objects.get(name=cust_id_type) 
    
    new_icustomer, created = ICustomer.objects.get_or_create(
        user_profile=cust_userprofile,
        defaults = {    
            'customer_code':cust_code,
            'customer_type':cust_type,
            'is_business_approved':False,
            'date_of_birth':cust_dob,
            'kyc_idproof_document_type':kyc_idproof_document_type,
            'kyc_idproof_document_value':cust_id_value,
        }
    
    )

    if created:
        print("\tICustomer  created")
    else:
        print("\tICustomer  existed already")
        
        
    if cust_id_type == 'aadhar':
        new_icustomer.aadhar_number = cust_id_value
        new_icustomer.save()
    
    if Business.objects.filter(code=bus_code).exists():
        business = Business.objects.get(code=bus_code)
        new_icustomer.business=business
        new_icustomer.save()
        
    # Create wallet for each of the icustomer
    new_icustomer_wallet, created = ICustomerWallet.objects.get_or_create(
        customer=new_icustomer,
        defaults = {    
            'current_balance':0,
            'credit_limit':0
        }
    
    )  
    
    if created:
        print("\tWallet  created")
    else:
        print("\tWallet  existed already")
        
    print("=============================================")
    
    
  

SIVAKAMI ( CBE219308 ) Count: 1 
	user created
	user profile created
	ICustomer  created
	Wallet  created
Ragunathan.R ( CBE223235 ) Count: 2 
	user created
	user profile created
	ICustomer  created
	Wallet  created
G.SHAMUGAVADIVELAN  ( CBE127019 ) Count: 3 
	user created
	user profile created
	ICustomer  created
	Wallet  created


In [10]:
current_cust_code

'kul_11707343'

In [37]:
from collections import defaultdict
data = defaultdict(dict)

count = 0
for index, row in df.iterrows():
    count = count+1
    booth_code = str(row['SupplyPointCode'])
    if booth_code in data[booth_code]:
        print(booth_code)    
    else:
        data[booth_code]['tm500'] = 0 
        data[booth_code]['sm500'] = 0
        data[booth_code]['sm250'] = 0
        data[booth_code]['fcm500'] = 0
    
    data[booth_code]['tm500'] += int(row['TM500'])
    data[booth_code]['sm250'] += int(row['SM250'])
    data[booth_code]['sm500'] += int(row['SM500'])
    data[booth_code]['fcm500'] += int(row['FCM500'])
    
# print(data)

# print it back
for booth_code in data:
    print("{},{}, {}, {}, {}".format(booth_code, data[booth_code]['tm500'], data[booth_code]['sm250'], data[booth_code]['sm500'], data[booth_code]['fcm500']))
          
          


1813,480, 0, 0, 0
90,0, 0, 3, 0
423,0, 0, 3, 0
105,0, 0, 0, 15
49,0, 0, 2, 0
72,0, 0, 2, 0
1766,0, 0, 20, 0
437,0, 1, 0, 0
1519,0, 0, 1, 0
76,0, 0, 17, 0
69,0, 0, 2, 0
809,0, 2, 1, 0
386,0, 0, 10, 0
368,0, 0, 4, 0
25,0, 0, 10, 0
108,0, 0, 14, 0
14,0, 0, 1, 0
92,0, 0, 2, 0
240,0, 1, 1, 0
250,0, 0, 0, 2
296,0, 0, 2, 0
447,0, 0, 10, 0
481,0, 0, 10, 0
514,0, 0, 3, 0
137,0, 0, 3, 0
273,0, 0, 4, 0
872,0, 0, 2, 1
1858,0, 0, 1, 0
33,0, 0, 2, 0
153,0, 0, 2, 0
274,0, 0, 2, 0
369,0, 0, 3, 0
406,0, 0, 3, 0
494,0, 0, 10, 0
495,0, 0, 4, 0
189,0, 1, 1, 0
99,0, 0, 2, 0
328,0, 0, 4, 0
438,0, 0, 1, 0
316,0, 0, 1, 0
26,0, 1, 1, 0
9,0, 0, 2, 0
96,0, 0, 6, 0
203,0, 0, 5, 0
325,0, 0, 3, 0
56,0, 3, 2, 0
470,0, 0, 3, 0
1,0, 1, 2, 0
64,0, 0, 1, 0
1800,0, 1, 1, 0
63,0, 0, 2, 0
237,0, 1, 1, 0
378,0, 2, 0, 0
439,0, 0, 3, 0
260,0, 0, 0, 2
362,0, 0, 1, 0
1082,0, 0, 5, 0
2619,0, 0, 4, 0
75,0, 0, 5, 0
123,0, 2, 1, 0
142,0, 0, 2, 0
11,0, 0, 10, 0
12,0, 0, 2, 0
40,0, 0, 4, 0
46,0, 0, 2, 0
47,0, 0, 2, 0
55,0, 0, 2, 0
79